# **VGG16**

VGG-16 Architecture의 구성

* 13 Convolution Layers + 3 Fully-connected Layers

* 3x3 convolution filters

* stride: 1 & padding: 1

* 2x2 max pooling (stride : 2)

* ReLU

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

In [4]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [5]:
# Preprocess image
train_images = preprocess_input(train_images)
test_images = preprocess_input(test_images)

In [6]:
# Preprocess label
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [7]:
# Load VGG-16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [8]:
# No parameter update in base model
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Add layer for CIFAR-10 classification
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [11]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Fit the model
model.fit(train_images, train_labels, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
625/625 [==============================] - 618s 987ms/step - loss: 2.2730 - accuracy: 0.5625 - val_loss: 1.1882 - val_accuracy: 0.6148
Epoch 2/5
625/625 [==============================] - 613s 981ms/step - loss: 0.8939 - accuracy: 0.6955 - val_loss: 1.1211 - val_accuracy: 0.6379
Epoch 3/5
625/625 [==============================] - 615s 985ms/step - loss: 0.6935 - accuracy: 0.7609 - val_loss: 1.1915 - val_accuracy: 0.6381
Epoch 4/5
625/625 [==============================] - 611s 978ms/step - loss: 0.5787 - accuracy: 0.7950 - val_loss: 1.2307 - val_accuracy: 0.6474
Epoch 5/5
625/625 [==============================] - 594s 951ms/step - loss: 0.4970 - accuracy: 0.8256 - val_loss: 1.3534 - val_accuracy: 0.6398


In [13]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_acc:.4f}')

313/313 [==============================] - 124s 395ms/step - loss: 1.3696 - accuracy: 0.6338
Test loss: 1.3696
Test accuracy: 0.6338
